In [1]:
import os
from pathlib import Path

from tensorboardX import SummaryWriter


from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.utils import common_utils

import datetime

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = {'cfg_file': '../cfgs/kitti_models/my_pointrcnn.yaml', 'batch_size': 1, 'epochs': None, 'workers': 1, 'extra_tag': 'default', 'ckpt': None, 'pretrained_model': None, 'launcher': 'none', 'tcp_port': 18888, 'sync_bn': False, 'fix_random_seed': False, 'ckpt_save_interval': 1, 'local_rank': 0, 'max_ckpt_save_num': 30, 'merge_all_iters_to_one_epoch': False, 'set_cfgs': None, 'max_waiting_mins': 0, 'start_epoch': 0, 'save_to_file': False}

args = Struct(**args)
print(args.cfg_file)


args.cfg_file= "./my_pointrcnn.yaml"
print(args.cfg_file)
cfg_from_yaml_file(args.cfg_file, cfg)
cfg.TAG = Path(args.cfg_file).stem
cfg.EXP_GROUP_PATH = '/'.join(args.cfg_file.split('/')[1:-1])  # remove 'cfgs' and 'xxxx.yaml'
if args.set_cfgs is not None:
    cfg_from_list(args.set_cfgs, cfg)


if args.launcher == 'none':
    dist_train = False
    total_gpus = 1
else:
    total_gpus, cfg.LOCAL_RANK = getattr(common_utils, 'init_dist_%s' % args.launcher)(
        args.tcp_port, args.local_rank, backend='nccl'
    )
    dist_train = True

if args.batch_size is None:
    args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
else:
    assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
    args.batch_size = args.batch_size // total_gpus

args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs

if args.fix_random_seed:
    common_utils.set_random_seed(666)

output_dir = cfg.ROOT_DIR / 'output' / cfg.EXP_GROUP_PATH / cfg.TAG / args.extra_tag
ckpt_dir = output_dir / 'ckpt'
output_dir.mkdir(parents=True, exist_ok=True)
ckpt_dir.mkdir(parents=True, exist_ok=True)

log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)

# log to file
logger.info('**********************Start logging**********************')
gpu_list = os.environ['CUDA_VISIBLE_DEVICES'] if 'CUDA_VISIBLE_DEVICES' in os.environ.keys() else 'ALL'
logger.info('CUDA_VISIBLE_DEVICES=%s' % gpu_list)

if dist_train:
    logger.info('total_batch_size: %d' % (total_gpus * args.batch_size))
for key, val in vars(args).items():
    logger.info('{:16} {}'.format(key, val))
log_config_to_file(cfg, logger=logger)
if cfg.LOCAL_RANK == 0:
    os.system('cp %s %s' % (args.cfg_file, output_dir))

tb_log = SummaryWriter(log_dir=str(output_dir / 'tensorboard')) if cfg.LOCAL_RANK == 0 else None

from pcdet.datasets import build_dataloader

train_set, train_loader, train_sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        class_names=cfg.CLASS_NAMES,
        batch_size=args.batch_size,
        dist=dist_train, workers=args.workers,
        logger=logger,
        training=True,
        merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
        total_epochs=args.epochs
    )


2020-08-19 04:32:09,303   INFO  **********************Start logging**********************
2020-08-19 04:32:09,305   INFO  CUDA_VISIBLE_DEVICES=ALL
2020-08-19 04:32:09,305   INFO  cfg_file         ./my_pointrcnn.yaml
2020-08-19 04:32:09,306   INFO  batch_size       1
2020-08-19 04:32:09,307   INFO  epochs           80
2020-08-19 04:32:09,308   INFO  workers          1
2020-08-19 04:32:09,308   INFO  extra_tag        default
2020-08-19 04:32:09,309   INFO  ckpt             None
2020-08-19 04:32:09,312   INFO  pretrained_model None
2020-08-19 04:32:09,312   INFO  launcher         none
2020-08-19 04:32:09,313   INFO  tcp_port         18888
2020-08-19 04:32:09,314   INFO  sync_bn          False
2020-08-19 04:32:09,314   INFO  fix_random_seed  False
2020-08-19 04:32:09,315   INFO  ckpt_save_interval 1
2020-08-19 04:32:09,315   INFO  local_rank       0
2020-08-19 04:32:09,316   INFO  max_ckpt_save_num 30
2020-08-19 04:32:09,316   INFO  merge_all_iters_to_one_epoch False
2020-08-19 04:32:09,31

../cfgs/kitti_models/my_pointrcnn.yaml
./my_pointrcnn.yaml


In [16]:
# %matplotlib
# wx, gtk, osx, tk, empty uses default
%matplotlib qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

colors = {
    'Car': 'b',
    'Tram': 'r',
    'Cyclist': 'g',
    'Van': 'c',
    'Truck': 'm',
    'Pedestrian': 'y',
    'Sitter': 'k'
}
axes_limits = [
    [-20, 80], # X axis range
    [-20, 20], # Y axis range
    [-3, 10]   # Z axis range
]
axes_str = ['X', 'Y', 'Z']

points=0.2
pp=train_set[100]['points']
points_step = int(1. / points)
point_size = 0.01 * (1. / points)
velo_range = range(0, pp.shape[0], points_step)

def draw_point_cloud(ax, title, axes=[0, 1, 2], xlim3d=None, ylim3d=None, zlim3d=None):
        """
        Convenient method for drawing various point cloud projections as a part of frame statistics.
        """
        ax.scatter(*np.transpose(pp[:, axes]), s=point_size, c=pp[:, 3], cmap='gray')
        ax.set_title(title)
        ax.set_xlabel('{} axis'.format(axes_str[axes[0]]))
        ax.set_ylabel('{} axis'.format(axes_str[axes[1]]))
        if len(axes) > 2:
            ax.set_xlim3d(*axes_limits[axes[0]])
            ax.set_ylim3d(*axes_limits[axes[1]])
            ax.set_zlim3d(*axes_limits[axes[2]])
            ax.set_zlabel('{} axis'.format(axes_str[axes[2]]))
        else:
            ax.set_xlim(*axes_limits[axes[0]])
            ax.set_ylim(*axes_limits[axes[1]])
        # User specified limits
        if xlim3d!=None:
            ax.set_xlim3d(xlim3d)
        if ylim3d!=None:
            ax.set_ylim3d(ylim3d)
        if zlim3d!=None:
            ax.set_zlim3d(zlim3d)


f2 = plt.figure(figsize=(15, 8))
ax2 = f2.add_subplot(111, projection='3d')
draw_point_cloud(ax2, 'Velodyne scan', xlim3d=(-10,30))
plt.show()